In [9]:
import numpy as np
import logging
import os
import xarray as xr
import pandas as pd
import holoviews as hv
from holoviews import opts
os.getcwd()

'C:\\Users\\Johannes\\PycharmProjects\\phd_base\\analysis'

In [10]:
import sys
sys.path.append("../pylim")
from pylim.helpers import get_path, make_dir
from pylim import smart
from pylim import reader
from pylim.bacardi import fdw_attitude_correction

log = logging.getLogger(__name__)
log.addHandler(logging.StreamHandler())
log.setLevel(logging.INFO)

# 20210715 Spain flight for contrail outbreak and Radiation Square candidate

In [11]:
print("20210715 SMART fixed - contrail outbreak over Spain - Radiation Square")
rs_start = pd.Timestamp(2021, 7, 15, 8, 0)
rs_end = pd.Timestamp(2021, 7, 15, 10, 0)
flight = "Flight_20210715a"
bacardi_dir = get_path("bacardi", flight)
smart_dir = get_path("calibrated", flight)
libradtran_dir = get_path("libradtran", flight)
sat_dir = get_path("satellite", flight)
if os.getcwd().startswith("C:"):
    outpath = f"C:/Users/Johannes/Documents/Doktor/campaigns/CIRRUS-HL/case_studies/{flight}"
else:
    outpath = f"/projekt_agmwend/home_rad/jroettenbacher/case_studies/{flight}"
make_dir(outpath)
ylims = (600, 1000)  # set ylims for all irradiance plots

20210715 SMART fixed - contrail outbreak over Spain - Radiation Square


### Find files and read them in

In [12]:
bahamas = reader.read_bahamas(flight)
bahamas_rs = bahamas.sel(time=slice(rs_start, rs_end))  # select subset of radiation square
smart_fdw_vnir_file = [f for f in os.listdir(smart_dir) if "Fdw_VNIR" in f][0]
smart_fdw_vnir = reader.read_smart_cor(smart_dir, smart_fdw_vnir_file)
smart_fdw_swir_file = [f for f in os.listdir(smart_dir) if "Fdw_SWIR" in f][0]
smart_fdw_swir = reader.read_smart_cor(smart_dir, smart_fdw_swir_file)
fdw_sim_file = [f for f in os.listdir(libradtran_dir) if "smart" in f][0]
fdw_sim = xr.open_dataset(f"{libradtran_dir}/{fdw_sim_file}")
fdw_sim_rs = fdw_sim.sel(time=slice(rs_start, rs_end))

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html
http://xarray.pydata.org/en/stable/user-guide/io.html

### Integrate SMART measurements

In [ ]:
smart_fdw_vnir_int = smart_fdw_vnir.sum(axis=1)
smart_fdw_swir_int = smart_fdw_swir.sum(axis=1, skipna=False)
smart_fdw = smart_fdw_vnir_int + smart_fdw_swir_int
smart_fdw_rs = smart_fdw.loc[slice(rs_start, rs_end)]

### Filter high roll angles

In [ ]:
roll_filter = np.abs(bahamas_rs["IRS_PHI"]) < 1
bahamas_rs_filtered = bahamas_rs.where(roll_filter)  # select only sections with roll < 1°
bahamas_inp = bahamas_rs.interp(time=smart_fdw_rs.index)  # interpolate bahamas on SMART time
roll_filter_smart = np.abs(bahamas_inp["IRS_PHI"]) < 1  # create filter for smart data
smart_fdw_rs_filtered = smart_fdw_rs.where(roll_filter_smart)

### Attitude correct F_dw with no offsets

In [ ]:
smart_fdw_df = pd.DataFrame(smart_fdw).rename(columns={0: "F_down"})  # make dataframe from series
# interpolate libradtran simulation and bahamas on smart time
fdw_sim_inp = fdw_sim.interp(time=smart_fdw.index, kwargs=dict(fill_value="extrapolate"))
bahamas_inp = bahamas.interp(time=smart_fdw.index, kwargs=dict(fill_value="extrapolate"))
F_down_att_no_offset, factor = fdw_attitude_correction(smart_fdw_df["F_down"].values,
                                                       roll=bahamas_inp.IRS_PHI.values,
                                                       pitch=-bahamas_inp.IRS_THE.values,
                                                       yaw=bahamas_inp.IRS_HDG.values, sza=fdw_sim_inp.sza.values,
                                                       saa=fdw_sim_inp.saa.values,
                                                       fdir=fdw_sim_inp.direct_fraction.values,
                                                       r_off=0, p_off=0)
smart_fdw_df["F_down_att_no"] = F_down_att_no_offset
# select radiation square
smart_fdw_df_rs = smart_fdw_df.loc[slice(rs_start, rs_end)]
smart_fdw_df_rs_filtered = smart_fdw_df_rs.loc[roll_filter_smart.values]

### Function to attitude correct with roll and pitch offset

In [ ]:
# all cells above have to be run, as this function uses not only lcoal variables! (This is bad coding but works here)
def att_corr(roll_offset, pitch_offset):
    F_down_att, factor = fdw_attitude_correction(smart_fdw_df["F_down"].values,
                                                 roll=bahamas_inp.IRS_PHI.values, pitch=-bahamas_inp.IRS_THE.values,
                                                 yaw=bahamas_inp.IRS_HDG.values, sza=fdw_sim_inp.sza.values,
                                                 saa=fdw_sim_inp.saa.values, fdir=fdw_sim_inp.direct_fraction.values,
                                                 r_off=roll_offset, p_off=pitch_offset)

    smart_fdw_df["F_down_att"] = F_down_att
    smart_fdw_df_rs = smart_fdw_df.loc[slice(rs_start, rs_end)]
    smart_fdw_df_rs_filtered = smart_fdw_df_rs.loc[roll_filter_smart.values]
    return hv.Curve(smart_fdw_df_rs_filtered["F_down_att"])

In [ ]:
def diff_per(roll_offset, pitch_offset):
    F_down_att, factor = fdw_attitude_correction(smart_fdw_df["F_down"].values,
                                                 roll=bahamas_inp.IRS_PHI.values, pitch=-bahamas_inp.IRS_THE.values,
                                                 yaw=bahamas_inp.IRS_HDG.values, sza=fdw_sim_inp.sza.values,
                                                 saa=fdw_sim_inp.saa.values, fdir=fdw_sim_inp.direct_fraction.values,
                                                 r_off=roll_offset, p_off=pitch_offset)

    smart_fdw_df["F_down_att"] = F_down_att
    smart_fdw_df_rs = smart_fdw_df.loc[slice(rs_start, rs_end)]
    fdw_diff = smart_fdw_df_rs.F_down_att - fdw_sim_inp["fdw"].sel(time=slice(rs_start, rs_end))
    fdw_diff_pc = np.abs(fdw_diff / smart_fdw_df_rs.F_down_att) * 100
    fdw_diff_pc_filtered = fdw_diff_pc.loc[roll_filter_smart.values]
    return hv.Curve(fdw_diff_pc_filtered)

### Create curves for plotting

In [ ]:
simulation = hv.Curve(fdw_sim_rs.fdw, label="Clear Sky Simulation").opts(color="purple", line_width=4)
no_offset = hv.Curve(smart_fdw_df_rs_filtered.F_down_att_no, label="No offset").opts(color="green", line_width=4)
dmap = hv.DynamicMap(att_corr, kdims=['roll_offset', 'pitch_offset']).opts(color="red", line_width=3)
dmap = dmap.redim.range(roll_offset=(-3, 3.0), pitch_offset=(-5, 5.0))

In [ ]:
overlay = simulation * no_offset * dmap

In [ ]:
%%opts Overlay [fontsize={'title':14, 'xlabel':16, 'ylabel':16, 'ticks':12}, xlabel="Time (UTC)", ylabel="Downward Irradiance (W m-2)"]
hv.output(widget_location='bottom')
overlay.opts(height=400, width=900, show_grid=True, show_legend=True, legend_position="bottom")


## Difference between corrected measurement and simulation

In [ ]:
dmap = hv.DynamicMap(diff_per, kdims=['roll_offset', 'pitch_offset']).opts(color="red", line_width=3)
dmap = dmap.redim.range(roll_offset=(-3, 3.0), pitch_offset=(-5, 5.0))
dmap.opts(height=400, width=900, show_grid=True, show_legend=True, ylabel="Difference (%)", xlabel="Time (UTC)", fontsize={'title':14, 'xlabel':16, 'ylabel':16, 'ticks':12})